# N-dimensional scans

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

from bluesky import RunEngine
from bluesky.plans import inner_product_scan, outer_product_scan, scan_nd
from bluesky.callbacks import LiveTable, LivePlot

# Set up simulated hardware.
from bluesky.examples import det4, motor1, motor2, motor3
# The 'det4' example detector a 2D Gaussian function of motor1, motor2.

RE = RunEngine({})
table = LiveTable(['det', 'motor'])

## Data Acquisition

### "Inner Product Scan": move motors together

In [0]:
# Move motor1 from 1-5 while moving motor2 from 10-50 -- both in 5 steps.
RE(inner_product_scan([det4], 5,
                      motor1, 1, 5,
                      motor2, 10, 50),
   LiveTable(['det4', 'motor1', 'motor2']))

As we did for `scan` in a previous notebook, we can inspect the plan without executing it.

In [0]:
from bluesky.plan_tools import print_summary

print_summary(inner_product_scan([det4], 5, motor1, 1, 5, motor2, 10, 50))

We can also visualize the plan's trajectory (again, without actually moving any motors).

In [0]:
from bluesky.plan_tools import plot_raster_path

plot_raster_path(inner_product_scan([det4], 5, motor1, 1, 20, motor2, 10, 50),
                 x_motor='motor1',
                 y_motor='motor2');

### "Outer Product Scan": move motors in a mesh

In [0]:
# Move motor1 from 1-3 in 3 steps and motor2 from 10-50 in 5 steps.
RE(outer_product_scan([det4],
                      motor1, 1, 30, 3,
                      motor2, 10, 50, 5, False),
   LiveTable(['det', 'motor1', 'motor2']))

The last parameter passed to ``outer_product_scan`` (``False``) controls whether the trajectory is "snaked" or not. Observe the difference when we visualize the trajectory:

In [0]:
from bluesky.plan_tools import plot_raster_path

plot_raster_path(outer_product_scan([det4], motor1, 1, 30, 3, motor2, 10, 50, 5, False),
                 x_motor='motor1',
                 y_motor='motor2');

In [0]:
from bluesky.plan_tools import plot_raster_path

plot_raster_path(outer_product_scan([det4], motor1, 1, 30, 3, motor2, 10, 50, 5, True),
                 x_motor='motor1',
                 y_motor='motor2');

Use ``LiveGrid`` to visualize the detector readings as a heatmap.

In [0]:
from bluesky.callbacks import LiveGrid

RE(outer_product_scan([det4], motor1, -3, 3, 15, motor2, -5, 5, 15, False),
   LiveGrid((15, 15), 'det4'))

### The general case: mixing inner and outer

For example, move two motors together (inner product) in a mesh against a third motor. We introduce the ``cycler`` object (from an external library) which is handy for assembling arbitrary multi-motor trajectories.

In [0]:
from cycler import cycler

traj1 = cycler(motor1, [1, 2, 3])
traj2 = cycler(motor2, [10, 20, 30])

In [0]:
list(traj1)

In [0]:
list(traj1 + traj2)  # an "inner product" trajectory

In [0]:
list(traj1 * traj2)  # an "outer product" trajectory

In [0]:
traj3 = cycler(motor3, [100, 200, 300])

list((traj1 + traj2) * traj3)

Pass this whole thing to ``scan_nd``, along with a list of detectors.

In [0]:
RE(scan_nd([det4], (traj1 + traj2) * traj3),
   LiveTable(['det4', 'motor1', 'motor2', 'motor3']))

Under the hood, ``inner_product_scan`` and ``outer_product_scan`` use ``scan_nd``; they just assemble the cycler objects for you.

## Exercises

1. Above we demonstrated ``inner_product_scan`` and ``outer_product_scan`` in two dimensions. They extend to any number of dimensions. Try a 3D scan using ``motor1``, ``motor2``, and ``motor3``.
2. Play around with cyclers to make various multi-motor trajectories.